<a href="https://colab.research.google.com/github/codekali/Neural-SC-Desciptor/blob/master/firstTrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import json
from torch.utils.data import Dataset
dictionary={}
X,Y = [],[]
count=-1   
file=open("/content/drive/My Drive/Transformer/train/train.token.sbt")
for line in file:
  X.append(line)
  count+=1
count=0
file=open("/content/drive/My Drive/Transformer/train/train.token.nl")
for line in file:
  Y.append(line)
  count+=1
print(len(Y))
print(len(X))
X=X[0:1000]
Y=Y[0:1000]

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Transformer/train/train.token.sbt'

In [0]:
# You can even put this data load operation in your Dataset class
import torch
import json
from torch.utils.data import Dataset
dictionary={}
X_valid,Y_valid = [],[]
count=-1   
file=open("/content/drive/My Drive/Transformer/val/valid.token.sbt")
for line in file:
  X_valid.append(line)
  count+=1
count=0
file=open("/content/drive/My Drive/Transformer/val/valid.token.nl")
for line in file:
  Y_valid.append(line)
  count+=1
print(len(Y_valid))
print(len(X_valid))
X_valid=X_valid[0:100]
Y_valid=Y_valid[0:100]

20000
20000


In [0]:
print(len(Y))
print(len(X))

10
10


In [0]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  def __init__(self, X_item, Y_item):
    self.X_item=X_item
    self.Y_item=Y_item
    # No need for this
    self.device=device

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.X_item)

  def __getitem__(self, index):
    X = self.X_item[index]
    Y = self.Y_item[index]
    return X, Y

In [0]:
#import mydata
import torch
from torch.utils import data
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
#cudann.benchmark = True

# Parameters
params = {'batch_size': 5,
          'shuffle': True,
          'num_workers': 6}
max_epochs = 10

training_set = Dataset(X,Y)
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(X_valid,Y_valid)
validation_generator = data.DataLoader(validation_set, **params)

cuda


In [0]:
!pip install transformers

In [0]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch.optim as optim

tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model = model.to(device)
# optimizer should be defined outside training loop
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(15):
    trainloss=0
    validloss=0
    # Training
    for local_batch, local_labels in training_generator: 
        """Transfer local_batch and labels to GPU"""
        #local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        """ You can write the above statement as:
        local_batch=(x.to(device) for x in local_batch)
        local_labels=(x.to(device) for x in local_labels)
        But I have transfered them to the device after encoding them
        """
        optimizer.zero_grad()
        """Forward Function Implementation"""
        # batch_encode_plus also returns a mask, try passing that mask as well in the model.
        # Refer - https://github.com/huggingface/transformers/blob/a79a9e1241b9257bbce1d933fee47f3a8ff90797/examples/translation/t5/evaluate_wmt.py#L17
        # Also, put the encode function in dataset. Return encoded matrix values from dataset. Here, only take it to gpu.
        input_ids = tokenizer.batch_encode_plus(local_batch, return_tensors="pt",pad_to_max_length=True)
        label = tokenizer.batch_encode_plus(local_labels, return_tensors="pt",pad_to_max_length=True)
        outputs = model(input_ids=(input_ids['input_ids']).to(device), lm_labels=(label['input_ids']).to(device))
        loss, prediction_scores = outputs[:2]
        trainloss=loss
        #print(loss)
        """Forward Function Ends here"""
        """Loss and optimizer"""
        # No you shouldn't do model zero grad. This will reset model.
        # model.zero_grad()
        loss.backward()
        # If you define optimizer here, new optimizer will be declared every step - no training will happen.
        # optimizer = optim.SGD(model.parameters(), lr=0.01)
        # optimizer.zero_grad() should be before loss.backward. Ideally at the top of training loop. Before model.
        # optimizer.zero_grad()
        optimizer.step()


    """Validation"""
    with torch.set_grad_enabled(False):
        for local_batch, local_labels in validation_generator:
          input_ids = tokenizer.batch_encode_plus(local_batch, return_tensors="pt",pad_to_max_length=True)
          label = tokenizer.batch_encode_plus(local_labels, return_tensors="pt",pad_to_max_length=True)
          model.eval()
          outputs = model(input_ids=(input_ids['input_ids']).to(device), lm_labels=(label['input_ids']).to(device))
          loss, prediction_scores = outputs[:2]
          validloss=loss
    print("\nEpoch ", epoch, " completed!, Training LOSS is: ", trainloss, " Validation loss is: ", validloss)
    #print(loss, "\n\n")



Epoch  0  completed!, Training LOSS is:  tensor(20.8415, device='cuda:0', grad_fn=<NllLossBackward>)  Validation loss is:  tensor(10.0547, device='cuda:0')

Epoch  1  completed!, Training LOSS is:  tensor(11.9161, device='cuda:0', grad_fn=<NllLossBackward>)  Validation loss is:  tensor(16.9352, device='cuda:0')

Epoch  2  completed!, Training LOSS is:  tensor(18.0669, device='cuda:0', grad_fn=<NllLossBackward>)  Validation loss is:  tensor(19.1240, device='cuda:0')

Epoch  3  completed!, Training LOSS is:  tensor(20.4952, device='cuda:0', grad_fn=<NllLossBackward>)  Validation loss is:  tensor(19.1648, device='cuda:0')

Epoch  4  completed!, Training LOSS is:  tensor(16.0663, device='cuda:0', grad_fn=<NllLossBackward>)  Validation loss is:  tensor(16.6882, device='cuda:0')

Epoch  5  completed!, Training LOSS is:  tensor(12.5693, device='cuda:0', grad_fn=<NllLossBackward>)  Validation loss is:  tensor(17.0689, device='cuda:0')

Epoch  6  completed!, Training LOSS is:  tensor(22.4999, 

In [0]:
 model.eval()

In [0]:
from datetime import datetime as dt
runtime =  dt.now()
model_name = "/content/drive/My Drive/Transformer/models/" + now.strftime("%d/%m/%Y %H:%M") + ".pth"

torch.save(model.state_dict(), model_name)